In [1]:
import $ivy.`org.apache.spark::spark-sql:2.4.0`
import $ivy.`org.apache.spark::spark-sql-kafka-0-10:2.4.0`

import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)

import $ivy.$                                  

import $ivy.$                                             


import org.apache.log4j.{Level, Logger}


In [2]:
import org.apache.spark.sql._

val spark = {
  NotebookSparkSession.builder()
    .master("local[*]")
    .getOrCreate()
}

import spark.implicits._


Loading spark-stubs
Getting spark JARs


SLF4J: No SLF4J providers were found.
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See https://www.slf4j.org/codes.html#noProviders for further details.


Creating SparkSession


Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties


Spark UI

import org.apache.spark.sql._


spark: SparkSession = org.apache.spark.sql.SparkSession@677d4e95
import spark.implicits._


In [63]:
import org.apache.spark.sql.types._
import org.apache.spark.sql.Encoders

// add current context to the encoder scopes to be able to decode the stream with the case classes
org.apache.spark.sql.catalyst.encoders.OuterScopes.addOuterScope(this)

//Define case classes to use as schema base
case class CassandraTransaction(id: String, account: Option[String], day: Option[String], hash: Option[String]) 
case class DebeziumPayload(ts_ms: Option[String], op: Option[String], after: Option[CassandraTransaction])
case class DebeziumMessage(payload: DebeziumPayload)

//Convert case classes to schema using encoder: https://spark.apache.org/docs/2.1.2/api/java/org/apache/spark/sql/Encoders.html
val valueSchema = Encoders.product[DebeziumMessage].schema


schema.printTreeString()

root
 |-- key: string (nullable = true)
 |-- value: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- schema: string (nullable = true)
 |    |    |-- payload: struct (nullable = true)
 |    |    |    |-- ts_ms: string (nullable = true)
 |    |    |    |-- op: string (nullable = true)
 |    |    |    |-- after: struct (nullable = true)
 |    |    |    |    |-- id: integer (nullable = false)
 |    |    |    |    |-- account: string (nullable = true)
 |    |    |    |    |-- day: string (nullable = true)
 |    |    |    |    |-- hash: string (nullable = true)



import org.apache.spark.sql.types._

import org.apache.spark.sql.Encoders

// add current context to the encoder scopes to be able to decode the stream with the case classes

defined class CassandraTransaction
defined class DebeziumPayload
defined class DebeziumMessage
valueSchema: StructType = StructType(
  StructField(
    "payload",
    StructType(
      StructField("ts_ms", StringType, true, {}),
      StructField("op", StringType, true, {}),
      StructField(
        "after",
        StructType(
          StructField("id", StringType, true, {}),
          StructField("account", StringType, true, {}),
          StructField("day", StringType, true, {}),
          StructField("hash", StringType, true, {})
        ),
        true,
        {}
      )
    ),
    true,
    {}
  )
)

In [64]:
val df = spark
  .readStream
  .format("kafka")
  .option("kafka.bootstrap.servers", "kafka:9092")
  .option("subscribe", "test_prefix.testdb.transactions")
  .option("startingOffsets","earliest")

  // .option("startingOffsets", """{"test_prefix.testdb.transactions":{"0":23}}""")
  // .option("endingOffsets", """{"test_prefix.testdb.transactions":{"0":50}}""")
  .load()

df: DataFrame = [key: binary, value: binary ... 5 more fields]

In [65]:
import org.apache.spark.sql.functions._
import org.apache.spark.sql.functions.from_json

val structuredDf = df
    .select(from_json(col("value").cast("string"), valueSchema).as("value"))

import org.apache.spark.sql.functions._

import org.apache.spark.sql.functions.from_json


structuredDf: DataFrame = [value: struct<payload: struct<ts_ms: string, op: string ... 1 more field>>]

In [66]:
// cast the dataset to typed values
val debeziumDf = structuredDf
    .select($"value.payload").as[DebeziumMessage]

debeziumDf: Dataset[DebeziumMessage] = [payload: struct<ts_ms: string, op: string ... 1 more field>]

In [69]:

val consoleOutput1 = debeziumDf
     .map(v => {
        kernel.publish.html(s"Got item <b>#$v</b>")
        v 
     })
     .writeStream
     .outputMode("append")
     .format("console")
     .start()

// wait to process the stream
Thread.sleep(5000)

// finish processing not to freez the kernel
consoleOutput1.stop()
// df.show()

start at cmd68.sc:8

1 / 1

Got item #DebeziumMessage(DebeziumPayload(Some(1670496912644),Some(i),Some(CassandraTransaction({"value":2,"deletion_ts":null,"set":true},Some({"value":"A1","deletion_ts":null,"set":true}),Some({"value":"d2","deletion_ts":null,"set":true}),Some({"value":"HV2","deletion_ts":null,"set":true})))))

Got item #DebeziumMessage(DebeziumPayload(Some(1670496912645),Some(i),Some(CassandraTransaction({"value":3,"deletion_ts":null,"set":true},Some({"value":"A1","deletion_ts":null,"set":true}),Some({"value":"d2","deletion_ts":null,"set":true}),Some({"value":"HV3","deletion_ts":null,"set":true})))))

Got item #DebeziumMessage(DebeziumPayload(Some(1670496912645),Some(i),Some(CassandraTransaction({"value":12,"deletion_ts":null,"set":true},Some({"value":"A4","deletion_ts":null,"set":true}),Some({"value":"d5","deletion_ts":null,"set":true}),Some({"value":"HV12","deletion_ts":null,"set":true})))))

...

Got item #DebeziumMessage(DebeziumPayload(Some(1670502892976),Some(i),Some(CassandraTransaction({"value":10,"deletion_ts":null,"set":true},Some({"value":"A4","deletion_ts":null,"set":true}),Some({"value":"d4","deletion_ts":null,"set":true}),Some({"value":"HV10","deletion_ts":null,"set":true})))))

Got item #DebeziumMessage(DebeziumPayload(Some(1670502892977),Some(i),Some(CassandraTransaction({"value":11,"deletion_ts":null,"set":true},Some({"value":"A4","deletion_ts":null,"set":true}),Some({"value":"d5","deletion_ts":null,"set":true}),Some({"value":"HV11","deletion_ts":null,"set":true})))))

Got item #DebeziumMessage(DebeziumPayload(Some(1670502892978),Some(i),Some(CassandraTransaction({"value":12,"deletion_ts":null,"set":true},Some({"value":"A4","deletion_ts":null,"set":true}),Some({"value":"d5","deletion_ts":null,"set":true}),Some({"value":"HV12","deletion_ts":null,"set":true})))))

-------------------------------------------
Batch: 0
-------------------------------------------
+--------------------+
|             payload|
+--------------------+
|[1670496912644, i...|
|[1670496912645, i...|
|[1670496912645, i...|
|[1670496912646, i...|
|[1670496912646, i...|
|[1670496912647, i...|
|[1670496912647, i...|
|[1670496912648, i...|
|[1670496912648, i...|
|[1670496912649, i...|
|[1670496912649, i...|
|[1670496912649, i...|
|[1670502892971, i...|
|[1670502892972, i...|
|[1670502892972, i...|
|[1670502892973, i...|
|[1670502892974, i...|
|[1670502892974, i...|
|[1670502892975, i...|
|[1670502892975, i...|
+--------------------+
only showing top 20 rows



consoleOutput1: streaming.StreamingQuery = org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@672641c